In [3]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
num_augmented_per_image = 4
original_folder = 'main'
mask_folder = 'mask'
augmented_folder = 'pp'
augmented_mask_folder = 'pm'
file_list = os.listdir(original_folder)

# Create an instance of the ImageDataGenerator for original images
datagen = ImageDataGenerator(
    rotation_range=30,  # Rotate images by 30 degrees
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create another instance of the ImageDataGenerator for masks
mask_datagen = ImageDataGenerator(
    rotation_range=30,  # Rotate masks by 30 degrees
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# List of rotation angles
rotation_angles = [30, 45, 60, 75,90,120,180,240,360]

# Iterate over each file in the original folder
for filename in file_list:
    original_path = os.path.join(original_folder, filename)
    
    # Assume mask files have the same name but with .png extension
    mask_filename = os.path.splitext(filename)[0] + '.png'
    mask_path = os.path.join(mask_folder, mask_filename)
    
    # Check if the mask file exists
    if not os.path.exists(mask_path):
        print(f"Mask file does not exist for {filename}: {mask_path}")
        continue

    # Load the original image
    img = load_img(original_path)
    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)

    # Load the corresponding mask
    mask = load_img(mask_path, color_mode="grayscale")
    y = img_to_array(mask)
    y = y.reshape((1,) + y.shape)

    # Set the same random seed for both generators
    seed = np.random.randint(0, 100000)

    # Generate augmented images and masks with rotation
    for angle in rotation_angles:
        # Set rotation range for both image and mask
        datagen.rotation_range = angle
        mask_datagen.rotation_range = angle
        
        # Generate exactly 10 augmented images and save them to the augmented folder
        img_gen = datagen.flow(x, batch_size=1, seed=seed)
        mask_gen = mask_datagen.flow(y, batch_size=1, seed=seed)

        for i in range(num_augmented_per_image):
            augmented_img = img_gen.next()[0]
            augmented_img = array_to_img(augmented_img)

            augmented_mask = mask_gen.next()[0]
            augmented_mask = array_to_img(augmented_mask)

            # Create a filename with the format:
            formatted_filename = f'{angle}_{i:05d}_{filename}'
            augmented_img.save(os.path.join(augmented_folder, formatted_filename))

            # Save the augmented mask with the .png extension
            mask_filename_aug = os.path.splitext(formatted_filename)[0] + '.png'
            augmented_mask.save(os.path.join(augmented_mask_folder, mask_filename_aug))